In [1]:
import os                       # For working in different directories
import numpy as np              # Data Libraries
import pandas as pd             # Data Libraries
import matplotlib.pyplot as plt # Visualization Library
import seaborn as sns           # Visualization Library
import geopandas as gpd         # geospatial library 
import requests                 # GET/Web Requests
from bs4 import BeautifulSoup   # Web scraping
import yaml                     # working w/ config file
from importnb import imports    # load other Jupyter Notebooks inside of this notebook
from functools import reduce
import pickle

In [2]:
# display entire column
from pandas import option_context

In [3]:
def load_api_key(directory):
    """Function to load API key w/out explicitily providing it in this public-facing file."""
    path = directory
    os.chdir(path)
    with open('config.yaml', 'r') as file:
        file_contents = yaml.safe_load(file)
    
    return file_contents['api_key'] 


api_key = load_api_key('/home/jcorley60/Documents/project_portfolio/DenverCraftBreweries/\
data/US Census Bureau')

# Total Population & Demographics
US Census Bureau program **American Community Survey (ACS)** provides 'vital information' annually 
* https://www.census.gov/programs-surveys/acs/about.html
* https://censusreporter.org/topics/table-codes/
* `CP02 Comparison Profile` [or `DP02` Data Profile] contains vital demographic information
    - see "variableMapping_acs_2021_5yr_data_dictionary.xlsx" for details
        - this is an Excel workbook created by webscraping a Census Bureau webpage dedicated to data profile codes
        - see function `get_census_bureau_key_data_dictionary` below
     
    
* The interactive tables/profiles found at https://data.census.gov were used to determine features/profiles amenable to this analysis
    
    
    
    
    
    
* https://www.census.gov/programs-surveys/acs/technical-documentation/table-shells.2021.html#list-tab-79594641
    - "Table List" - Download information about ACS tables, including table IDs, titles, universes, as well as 1-year/5-year availability, in one spreadsheet.
* **ACS API**: https://www.census.gov/data/developers/data-sets/acs-5year.html
    - enables grabbing demographic *Comparison Profiles* vs manually downloading individual features

### Get Available Data Profiles from US Census Bureau
* Census Bureau **Data Profiles** package & present Census Profiles together for end-user consumption
* Several DP profiles exist state-wide in 2021 for Colorado from the 5-Year American Community Survey:
    - `CP02` - Comparative Social Characteristics in the US
    - `CP03` - Comparative Economic Characteristics
    - `CP04` - Comparative Housing Characteristics
    - `CP05` - Comparative Demographic Estimates

#### Get Census Bureau variable keys to decipher each Comparative Profile/Data Profile

In [4]:
def get_census_bureau_key_data_dictionary(url):
    """Function to send GET request to Census data dictionary website for the 2021 ACS 5-year 2021.
    params: url - Census Bureau website which houses data dictionary for data profiles
    returns:
    BeautifulSoup [soup] object from webpage which will need to be cleaned/parsed
    """
    response = requests.get(url)
    try:
        if response.status_code == 200:
            print(f"{response.status_code} success")
        soup = BeautifulSoup(response.content)
        return soup
    except:
        print("Unable to process -- exception encountered")
        
        
soup = get_census_bureau_key_data_dictionary('https://api.census.gov/data/2021/acs/acs5/profile/variables.html')

200 success


### Save soup data dictionary to file in event site goes offline/project archiving

In [5]:
os.chdir(
    '/home/jcorley60/Documents/project_portfolio/DenverCraftBreweries/data/US Census Bureau/American Community Survey/backups'
)

# Save the soup object to file
with open("acs_data_dictionary_soup.pickle", "wb") as f:
    pickle.dump(soup, f)

# Read the soup object from a file
with open("acs_data_dictionary_soup.pickle", "rb") as f:
    soup_obj = pickle.load(f)

print(soup_obj.title)

<title>Census Data API: /data/2021/acs/acs5/profile/variables</title>


In [6]:
def parse_soup(soup_object, print_output):
    """Function to parse Beautiful Soup object for codes, labels, etc. to construct 
    the 2021 ACS 5-Year data dictionary.
    params: 
    soup_object{Beautiful Soup object}
    print_output{bool} - if True prints webscraping dictionary
    returns:
    data dictionary {dictionary}
    """
    parsed_table = {
        'code': [], 
        'label': [], 
        'concept': [], 
        'attributes': [], 
        'predicate_type': [], 
        'group': []
    }

    table_body = soup_object.find('tbody')
    for row in table_body.findAll('tr'):
        code = row.td.text
        label = row.findAll('td')[1].text
        concept = row.findAll('td')[2].text
        attributes = row.findAll('td')[4].text
        predicate_type = row.findAll('td')[6].text
        group = row.findAll('td')[7].text

        parsed_table['code'].append(code)
        parsed_table['label'].append(label)
        parsed_table['concept'].append(concept)
        parsed_table['attributes'].append(attributes)
        parsed_table['predicate_type'].append(predicate_type)
        parsed_table['group'].append(group)
        
        if print_output:
            print('****\n', row, '\n')
            print(row.findAll('td'))
            print(f"code = {code}")
            print(f"label = {label}")
            print(f"concept = {concept}")
            print(f"attributes = {attributes}")
            print(f"predicate_type = {predicate_type}")
            print(f"group = {group}")

    return parsed_table
            
    
df = pd.DataFrame(parse_soup(soup_object=soup, print_output=False))

#### Cleanup data dictionary dataframe

In [7]:
def clean_data_dictionary_dataframe(df):
    """Function to cleanup data dictionary dataframe:
        - remove whitespace, '\n' character
        - replace empty placeholder w/ empty character
    params: df{Pandas DataFrame} - df of parsed webscraping dictionary
    returns:
    df{Pandas DataFrame} - cleaned data dictionary Pandas dataframe
    """
    df['group'].replace('\n\t\t\t\t\t\tN/A\n\t\t\t\t\t', '', inplace=True)
    
    string = "DP04,\n                DP05,\n                DP02PR,\n                DP02,\n                DP03"
    df['group'][df['group'] == string] = df['group'][df['group'] == string].str.replace(" ", "").str.replace("\n", ' ')
    
    df['attributes'] = df['attributes'].replace(r'\s+|\\n', ' ', regex=True) 
    
    # remove '!!' separators to use ' :: ' instead for label column values which is easier on the eyes
    df['label'] = df['label'].str.split('!!').str.join(' :: ')
    
    df = df[['code', 'label', 'predicate_type', 'attributes', 'group', 'concept']]
    
    display(df['group'].value_counts())
    
    return df


df_data_dictionary = clean_data_dictionary_dataframe(df)
df_data_dictionary.iloc[15:20]

group
DP02                              308
DP02PR                            308
DP04                              286
DP03                              274
DP05                              178
                                   37
DP04, DP05, DP02PR, DP02, DP03      1
Name: count, dtype: int64

,code,label,predicate_type,attributes,group,concept
15,DP02_0002PE,Percent :: HOUSEHOLDS BY TYPE :: Total househo...,float,"DP02_0002PEA, DP02_0002PM, DP02_0002PMA",DP02,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...
16,DP02_0003E,Estimate :: HOUSEHOLDS BY TYPE :: Total househ...,int,"DP02_0003EA, DP02_0003M, DP02_0003MA",DP02,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...
17,DP02_0003PE,Percent :: HOUSEHOLDS BY TYPE :: Total househo...,float,"DP02_0003PEA, DP02_0003PM, DP02_0003PMA",DP02,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...
18,DP02_0004E,Estimate :: HOUSEHOLDS BY TYPE :: Total househ...,int,"DP02_0004EA, DP02_0004M, DP02_0004MA",DP02,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...
19,DP02_0004PE,Percent :: HOUSEHOLDS BY TYPE :: Total househo...,float,"DP02_0004PEA, DP02_0004PM, DP02_0004PMA",DP02,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...


#### Write `df_data_dictionary` to CSV for convenient reference and review
* a saved copy of this was modified to carry out the variable mapping `variableMapping_acs_2021_5yr_data_dictionary.xlsx` 
* ACS Data Profiles are data output for Comparative Profiles

In [8]:
os.chdir(
    '/home/jcorley60/Documents/project_portfolio/DenverCraftBreweries/data/US Census Bureau/American Community Survey'
)
df_data_dictionary.to_excel("acs_2021_5yr_data_dictionary.xlsx", index=False)

#### Read in modified version of `acs_2021_5yr_data_dictionary.xlsx` (see above) which maps features from Comparative Profiles
* we saved a copy of `use acs_2021_5yr_data_dictionary.xlsx` & added several columns which analyze the Excel Workbook output as the original file is overwritten every time `df_data_dictionary` above is written to file via method *.to_excel()*

In [9]:
def load_mapped_data_dictionary():
    """A saved copy of 'acs_2021_5yr_data_dictionary.xlsx' (see above) was modified to include 
    variable mappings, or which data profile variables have been chosen for our analysis.
    See `variableMapping_acs_2021_5yr_data_dictionary.xlsx` found in 
    '/DenverCraftBreweries/data/US Census Bureau/American Community Survey'
    returns: df_var_map{Pandas DataFrame} - DF of 'acs_2021_5yr_data_dictionary.xlsx' sheet 'variable mapping'
    """
    df_var_map = pd.read_excel('variableMapping_acs_2021_5yr_data_dictionary.xlsx', sheet_name='variable mapping')
    df_var_map = df_var_map[df_var_map['column name'].notna()]
    df_var_map.reset_index(inplace=True)
    df_var_map.drop(columns=[
        'index', 
        'column number', 
        'predicate_type', 
        'attributes', 
        'group', 
        'concept'
    ], axis=1, inplace=True)

    # temporarily display entire DF, only to immediately drop the unneeded column `label`
    with option_context('display.max_colwidth', None):
        display(df_var_map)

    df_var_map.drop(columns=['label'], axis=1, inplace=True)
    
    df_var_map.to_pickle(
        '/home/jcorley60/Documents/project_portfolio/DenverCraftBreweries/data/US Census Bureau/American Community Survey/backups/df_var_map.pkl'
    )
    
    return df_var_map


df_var_map = load_mapped_data_dictionary()

,column name,label,code,label.1
0,tot_hh,Total Households:,DP02_0001E,Estimate :: HOUSEHOLDS BY TYPE :: Total households
1,mar_hh,Married Couple Household:,DP02_0002E,Estimate :: HOUSEHOLDS BY TYPE :: Total households :: Married-couple household
2,mar_hh_%,Married Couple Household\n(% of total households):,DP02_0002PE,Percent :: HOUSEHOLDS BY TYPE :: Total households :: Married-couple household
3,mar_child,Married Couple Household w/ children <18,DP02_0003E,Estimate :: HOUSEHOLDS BY TYPE :: Total households :: Married-couple household :: With children of the householder under 18 years
4,mar_child_%,Married Couple Household w/ children <18\n(% of total households):,DP02_0003PE,Percent :: HOUSEHOLDS BY TYPE :: Total households :: Married-couple household :: With children of the householder under 18 years
...,...,...,...,...
149,target_age_%,"Target Ages, 21-64 (% of all ages):",NaN,NaN
150,target_hhinc,Target household incomes ≥$75K:,NaN,NaN
151,target_hhinc_%,Target household incomes ≥$75K (% of all incomes):,NaN,NaN
152,num_major_roads,Total number of major roads:,NaN,NaN


In [10]:
df_var_map['column name'].values

array(['tot_hh', 'mar_hh', 'mar_hh_%', 'mar_child', 'mar_child_%',
       'cohabit_hh', 'cohabit_hh_%', 'cohabit_child', 'cohabit_child_%',
       'hh_male', 'hh_male_%', 'hh_male_child', 'hh_male_child_%',
       'hh_male_solo', 'hh_male_solo_%', 'hh_female', 'hh_female_%',
       'hh_female_child', 'hh_female_child_%', 'hh_female_solo',
       'hh_female_solo_%', 'tot_hh_avg_hh_size', 'tot_hh_avg_fam_size',
       'pop_in_hh', 'pop_in_hh_%', 'edu_in_college', 'edu_in_college_%',
       'edu_attain_>=25', 'edu_hs', 'edu_hs_%', 'edu_some_col',
       'edu_some_col_%', 'edu_as', 'edu_as_%', 'edu_bs', 'edu_bs_%',
       'edu_grad_prof', 'edu_grad_prof_%', 'edu_>=_hs', 'edu_>=_hs_%',
       'edu_>=_bs', 'edu_>=_bs_%', 'veteran', 'veteran_%', 'diff_res_1yr',
       'diff_res_1yr_%', 'tot_pop_>=16', 'in_laborforce',
       'in_laborforce_%', 'notin_laborforce', 'notin_laborforce_%',
       'unemploy_rate', 'unemploy_rate_%', 'commute_wfh', 'commute_wfh_%',
       'commute_mean_mins', 'occ_>

### Get Comparative Profiles (CP data output called Data Profiles by Census Bureau)
* **GET** request: **<api.census.gov/data/2021/acs/acs5/profile?get=group(DP##)&for=state:08&key=YOUR_KEY_GOES_HERE>**
    - where DP## is the desired DP 2-digit profile number, related to the respective Comparative Profile (CP)
        - a Data Profile (DP) provides data for a Comparative Profile (CP)

In [11]:
def get_census_bureau_data_profiles(year, acs_interval, state_code):
    """Downloads all available Comparative Profiles (variables available for download together),
    concatenates them together by common geographic location.
    The most recent CPs/DPs available from the ACS for all geographies was the 5-year for 2021, and
    includes profiles 2, 3, 4, and 5 (4 total); the interactive website was used to determine
    available CP's.
    params:
    year{str} - American Community Survey (ACS) origin year
    acs_interval{str} - ACS 1 or 5 year interval
    state_code{str} - FIPS code identfying a US State
    returns:
    df_data_profiles{Pandas DataFrame} - concatenated data profiles as a Pandas dataframe
    """
    pd.set_option('display.max_columns', 50)
    
    if acs_interval == 5:
        acs_int = 'acs5'
    elif acs_interval == 1:
        acs_int = 'acs1'
        
    data_profile_nums = [f"0{i}" for i in range(2,6)]
    # total columns for dps: 4,200; w/ 4184 payload columns + 4 ID columns (GEO_ID, NAME, state, place)
    dps = []
    for num in data_profile_nums:
        url = f'https://api.census.gov/data/{year}/acs/{acs_int}/profile?get=group(DP{num})&for=place:*&in=state:{state_code}&key={api_key}'
        
        try:
            response = requests.get(url)
            data = pd.read_csv(url)

            df = pd.DataFrame(response.json())
            df = pd.DataFrame(df.values[1:], columns=df.iloc[0])
            
            print(f"DP{num}: {response.status_code} {'status code':.<15}{df.shape}")

            dps.append(df)
            
        except:
            return "issue encountered"
    
    # axis=1 for side-by-side columns but same rows/geographies
    df_data_profiles = pd.concat(dps, axis=1)
    
    # drop duplicate columns (columns GEO_ID, NAME, state, place would show up 4x each otherwise)
    df_data_profiles = df_data_profiles.loc[:,~df_data_profiles.columns.duplicated()].copy()
    
    return df_data_profiles
        

df_data_profiles = get_census_bureau_data_profiles(2021, 5, '08')
df_data_profiles

DP02: 200 status code....(482, 1236)
DP03: 200 status code....(482, 1100)
DP04: 200 status code....(482, 1148)
DP05: 200 status code....(482, 716)


,DP02_0001E,DP02_0001EA,DP02_0001M,DP02_0001MA,DP02_0001PE,DP02_0001PEA,DP02_0001PM,DP02_0001PMA,DP02_0002E,DP02_0002EA,DP02_0002M,DP02_0002MA,DP02_0002PE,DP02_0002PEA,DP02_0002PM,DP02_0002PMA,DP02_0003E,DP02_0003EA,DP02_0003M,DP02_0003MA,DP02_0003PE,DP02_0003PEA,DP02_0003PM,DP02_0003PMA,DP02_0004E,...,DP05_0086PMA,DP05_0087E,DP05_0087EA,DP05_0087M,DP05_0087MA,DP05_0087PE,DP05_0087PEA,DP05_0087PM,DP05_0087PMA,DP05_0088E,DP05_0088EA,DP05_0088M,DP05_0088MA,DP05_0088PE,DP05_0088PEA,DP05_0088PM,DP05_0088PMA,DP05_0089E,DP05_0089EA,DP05_0089M,DP05_0089MA,DP05_0089PE,DP05_0089PEA,DP05_0089PM,DP05_0089PMA
0,1040,None,132,None,1040,None,-888888888,(X),717,None,151,None,68.9,None,9.6,None,353,None,140,None,33.9,None,10.9,None,14,...,(X),2002,None,207,None,2002,None,-888888888,(X),1201,None,169,None,60.0,None,4.7,None,801,None,113,None,40.0,None,4.7,None
1,395,None,155,None,395,None,-888888888,(X),202,None,154,None,51.1,None,24.7,None,38,None,38,None,9.6,None,10.8,None,0,...,(X),790,None,353,None,790,None,-888888888,(X),328,None,116,None,41.5,None,16.2,None,462,None,302,None,58.5,None,16.2,None
2,271,None,82,None,271,None,-888888888,(X),56,None,35,None,20.7,None,12.8,None,0,None,13,None,0.0,None,12.9,None,32,...,(X),451,None,139,None,451,None,-888888888,(X),164,None,75,None,36.4,None,10.5,None,287,None,94,None,63.6,None,10.5,None
3,643,None,84,None,643,None,-888888888,(X),450,None,60,None,70.0,None,8.8,None,317,None,72,None,49.3,None,9.8,None,7,...,(X),5043,None,449,None,5043,None,-888888888,(X),3427,None,394,None,68.0,None,3.9,None,1616,None,220,None,32.0,None,3.9,None
4,763,None,96,None,763,None,-888888888,(X),345,None,74,None,45.2,None,7.8,None,114,None,41,None,14.9,None,4.6,None,45,...,(X),1451,None,170,None,1451,None,-888888888,(X),732,None,103,None,50.4,None,3.5,None,719,None,96,None,49.6,None,3.5,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,3224,None,281,None,3224,None,-888888888,(X),2633,None,283,None,81.7,None,6.0,None,902,None,204,None,28.0,None,5.8,None,142,...,(X),7026,None,699,None,7026,None,-888888888,(X),3797,None,444,None,54.0,None,2.3,None,3229,None,326,None,46.0,None,2.3,None
478,69,None,51,None,69,None,-888888888,(X),42,None,43,None,60.9,None,39.8,None,0,None,13,None,0.0,None,38.3,None,0,...,(X),113,None,86,None,113,None,-888888888,(X),69,None,51,None,61.1,None,20.7,None,44,None,45,None,38.9,None,20.7,None
479,934,None,132,None,934,None,-888888888,(X),437,None,90,None,46.8,None,8.3,None,215,None,70,None,23.0,None,7.6,None,91,...,(X),1534,None,200,None,1534,None,-888888888,(X),775,None,127,None,50.5,None,4.8,None,759,None,122,None,49.5,None,4.8,None
480,187,None,71,None,187,None,-888888888,(X),103,None,54,None,55.1,None,15.0,None,49,None,42,None,26.2,None,17.7,None,0,...,(X),336,None,117,None,336,None,-888888888,(X),194,None,65,None,57.7,None,8.1,None,142,None,65,None,42.3,None,8.1,None


#### Write Data Profiles to file for backup in case site goes down/project archiving

In [12]:
os.chdir(
    '/home/jcorley60/Documents/project_portfolio/DenverCraftBreweries/data/US Census Bureau/American Community Survey/backups'
)

df_data_profiles.to_pickle("2021_ACS_RAW_All_DataProfiles.pkl")

os.chdir(
    '/home/jcorley60/Documents/project_portfolio/DenverCraftBreweries/data/US Census Bureau/American Community Survey'
)

#### Concatenate Data Profiles (DPs) together by column (by shared geographic location/row)

In [13]:
def organize_data_profiles(dp_df):
    """Takes concatenated Census Bureau Data Profiles, as a dataframe and reorganizes columns
    such that the identifying columns occur at the beginning of a row, 
    instead of somewhere in the middle.
    Finds column headers which do not start w/ 'DP' for data profile variables.
    This function assumes the Census Bureau will list the following variables together 
    w/in their DP/CP output: ['GEO_ID', 'NAME', 'state', 'place']
    params: dp_df{Pandas DataFrame} - df containing a Census Bureau data profile (a comparative profile analog)
    returns:
    df_dps_merged{Pandas dataframe} - organized merged data profile df
    """
    min_idx, max_idx, idx_dict = np.Infinity, -np.Infinity, dict()
    
    # find columns which do not begin w/ 'DP...'
    id_cols = dp_df.columns[~dp_df.columns.str.contains('DP')].to_list()
    for col in id_cols:
        idx = dp_df.columns.get_loc(col)
        # add to dictionary
        idx_dict[col] = idx
        # find lowest index
        if idx < min_idx:
            min_idx = idx
        if idx > max_idx:
            max_idx = idx

    print('Index Locations:\n', idx_dict, '\nmin_idx:', min_idx, '\nmax_idx:', max_idx, sep='')

    df_data_profiles_col_start = dp_df.iloc[:, :idx_dict['GEO_ID']]
    df_data_profiles_col_id = dp_df.iloc[:, idx_dict['GEO_ID']:idx_dict['NAME']+1]  # 1232:1234
    df_data_profiles_col_end = dp_df.iloc[:, idx_dict['place']+1:]  # we don't need 'state' or 'place'

    df_dps_merged = pd.concat(
        [df_data_profiles_col_id, df_data_profiles_col_start, df_data_profiles_col_end], 
        axis=1
    )
    df_dps_merged.to_csv(
        "/home/jcorley60/Documents/project_portfolio/DenverCraftBreweries/data/US Census Bureau/American Community Survey/backups/DataProfiles_Merged.csv", 
        index=False
    )
    
    return df_dps_merged


df_dps_merged = organize_data_profiles(df_data_profiles)
df_dps_merged

Index Locations:
{'GEO_ID': 1232, 'NAME': 1233, 'state': 1234, 'place': 1235}
min_idx:1232
max_idx:1235


,GEO_ID,NAME,DP02_0001E,DP02_0001EA,DP02_0001M,DP02_0001MA,DP02_0001PE,DP02_0001PEA,DP02_0001PM,DP02_0001PMA,DP02_0002E,DP02_0002EA,DP02_0002M,DP02_0002MA,DP02_0002PE,DP02_0002PEA,DP02_0002PM,DP02_0002PMA,DP02_0003E,DP02_0003EA,DP02_0003M,DP02_0003MA,DP02_0003PE,DP02_0003PEA,DP02_0003PM,...,DP05_0086PMA,DP05_0087E,DP05_0087EA,DP05_0087M,DP05_0087MA,DP05_0087PE,DP05_0087PEA,DP05_0087PM,DP05_0087PMA,DP05_0088E,DP05_0088EA,DP05_0088M,DP05_0088MA,DP05_0088PE,DP05_0088PEA,DP05_0088PM,DP05_0088PMA,DP05_0089E,DP05_0089EA,DP05_0089M,DP05_0089MA,DP05_0089PE,DP05_0089PEA,DP05_0089PM,DP05_0089PMA
0,1600000US0800320,"Acres Green CDP, Colorado",1040,None,132,None,1040,None,-888888888,(X),717,None,151,None,68.9,None,9.6,None,353,None,140,None,33.9,None,10.9,...,(X),2002,None,207,None,2002,None,-888888888,(X),1201,None,169,None,60.0,None,4.7,None,801,None,113,None,40.0,None,4.7,None
1,1600000US0800620,"Aetna Estates CDP, Colorado",395,None,155,None,395,None,-888888888,(X),202,None,154,None,51.1,None,24.7,None,38,None,38,None,9.6,None,10.8,...,(X),790,None,353,None,790,None,-888888888,(X),328,None,116,None,41.5,None,16.2,None,462,None,302,None,58.5,None,16.2,None
2,1600000US0800760,"Aguilar town, Colorado",271,None,82,None,271,None,-888888888,(X),56,None,35,None,20.7,None,12.8,None,0,None,13,None,0.0,None,12.9,...,(X),451,None,139,None,451,None,-888888888,(X),164,None,75,None,36.4,None,10.5,None,287,None,94,None,63.6,None,10.5,None
3,1600000US0800870,"Air Force Academy CDP, Colorado",643,None,84,None,643,None,-888888888,(X),450,None,60,None,70.0,None,8.8,None,317,None,72,None,49.3,None,9.8,...,(X),5043,None,449,None,5043,None,-888888888,(X),3427,None,394,None,68.0,None,3.9,None,1616,None,220,None,32.0,None,3.9,None
4,1600000US0800925,"Akron town, Colorado",763,None,96,None,763,None,-888888888,(X),345,None,74,None,45.2,None,7.8,None,114,None,41,None,14.9,None,4.6,...,(X),1451,None,170,None,1451,None,-888888888,(X),732,None,103,None,50.4,None,3.5,None,719,None,96,None,49.6,None,3.5,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,1600000US0886117,"Woodmoor CDP, Colorado",3224,None,281,None,3224,None,-888888888,(X),2633,None,283,None,81.7,None,6.0,None,902,None,204,None,28.0,None,5.8,...,(X),7026,None,699,None,7026,None,-888888888,(X),3797,None,444,None,54.0,None,2.3,None,3229,None,326,None,46.0,None,2.3,None
478,1600000US0886200,"Woody Creek CDP, Colorado",69,None,51,None,69,None,-888888888,(X),42,None,43,None,60.9,None,39.8,None,0,None,13,None,0.0,None,38.3,...,(X),113,None,86,None,113,None,-888888888,(X),69,None,51,None,61.1,None,20.7,None,44,None,45,None,38.9,None,20.7,None
479,1600000US0886310,"Wray city, Colorado",934,None,132,None,934,None,-888888888,(X),437,None,90,None,46.8,None,8.3,None,215,None,70,None,23.0,None,7.6,...,(X),1534,None,200,None,1534,None,-888888888,(X),775,None,127,None,50.5,None,4.8,None,759,None,122,None,49.5,None,4.8,None
480,1600000US0886475,"Yampa town, Colorado",187,None,71,None,187,None,-888888888,(X),103,None,54,None,55.1,None,15.0,None,49,None,42,None,26.2,None,17.7,...,(X),336,None,117,None,336,None,-888888888,(X),194,None,65,None,57.7,None,8.1,None,142,None,65,None,42.3,None,8.1,None


#### Merge consolidated DPs (`data_profiles`) w/ selected variables from Data Profiles (`df_var_map`)
* there are 4,188 variables (plus 4 ID columns) vs 482 rows and so we can better minimize our `Data Profiles` by merging on only needed variables from `df_var_map` to remove unneeded columns/variables

In [14]:
def get_select_variables_from_consolidated_data_profiles(df_merged_data_profiles, df_variables):
    """This function will retrieve only the desired variables from the consolidated Data Profiles,
    as determined by the mapped data dictionary.  An important identifying column
    from the data profiles 'GEO_ID' is later used to merge w/ geospatial geometry in a GeoDataFrame.
    DP variables/columns are then renamed in this function for the end-user ease of use.
    params:
    df_merged_data_profiles{Pandas DataFrame} - df of desired data profiles
    df_variables{Pandas DataFrame} - df of data dictionary
    returns:
    select data profile variables along w/ identifying columns GEO_ID & NAME
    """
    code_list = df_variables['code'].to_list()
    # ensure identifying columns are included
    code_list.append("GEO_ID")
    code_list.append("NAME")

    df_demographics = df_merged_data_profiles[df_merged_data_profiles.columns.intersection(code_list)]
    
    # rename Data Profile variables using mapped data dictionary (presented as a DF)
    rename_dict = dict()
    for i in range(len(df_variables.to_dict('list')['code'])):
        rename_dict[df_variables.to_dict('list')['code'][i]] = df_variables.to_dict('list')['column name'][i]

    df_demographics.rename(columns=rename_dict, inplace=True)
    
    return df_demographics
    
    
df_demographics = get_select_variables_from_consolidated_data_profiles(df_dps_merged, df_var_map)
df_demographics

/tmp/ipykernel_400859/2625107065.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_demographics.rename(columns=rename_dict, inplace=True)


,GEO_ID,NAME,tot_hh,mar_hh,mar_hh_%,mar_child,mar_child_%,cohabit_hh,cohabit_hh_%,cohabit_child,cohabit_child_%,hh_male,hh_male_%,hh_male_child,hh_male_child_%,hh_male_solo,hh_male_solo_%,hh_female,hh_female_%,hh_female_child,hh_female_child_%,hh_female_solo,hh_female_solo_%,tot_hh_avg_hh_size,tot_hh_avg_fam_size,...,tot_pop_female_%,tot_pop_20-24,tot_pop_20-24_%,tot_pop_25-34,tot_pop_25-34_%,tot_pop_35-44,tot_pop_35-44_%,tot_pop_45-54,tot_pop_45-54_%,tot_pop_55-59,tot_pop_55-59_%,tot_pop_60-64,tot_pop_60-64_%,tot_pop_>=21,tot_pop_>=21_%,tot_pop_>=18,tot_pop_>=18_%,tot_pop_>=18_male,tot_pop_>=18_male_%,tot_pop_>=18_female,tot_pop_>=18_female_%,tot_pop_white,tot_pop_white_%,tot_pop_hisp,tot_pop_hisp_%
0,1600000US0800320,"Acres Green CDP, Colorado",1040,717,68.9,353,33.9,14,1.3,0,0.0,198,19.0,8,0.8,101,9.7,111,10.7,9,0.9,66,6.3,2.63,3.01,...,47.0,157,5.7,297,10.9,462,16.9,373,13.7,172,6.3,190,7.0,2067,75.7,2152,2152,1216,56.5,936,43.5,2221,81.3,63,2.3
1,1600000US0800620,"Aetna Estates CDP, Colorado",395,202,51.1,38,9.6,0,0.0,0,0.0,77,19.5,59,14.9,0,0.0,116,29.4,47,11.9,25,6.3,3.18,3.16,...,57.9,129,10.3,312,24.9,121,9.6,146,11.6,59,4.7,46,3.7,878,70.0,904,904,363,40.2,541,59.8,618,49.2,635,50.6
2,1600000US0800760,"Aguilar town, Colorado",271,56,20.7,0,0.0,32,11.8,25,9.2,37,13.7,0,0.0,26,9.6,146,53.9,0,0.0,129,47.6,1.76,2.71,...,63.1,51,10.7,8,1.7,35,7.3,70,14.7,40,8.4,53,11.1,415,87.0,451,451,164,36.4,287,63.6,322,67.5,323,67.7
3,1600000US0800870,"Air Force Academy CDP, Colorado",643,450,70.0,317,49.3,7,1.1,7,1.1,99,15.4,35,5.4,64,10.0,87,13.5,76,11.8,11,1.7,3.36,3.64,...,34.3,2727,44.2,429,7.0,375,6.1,178,2.9,74,1.2,6,0.1,2996,48.6,5174,5174,3516,68.0,1658,32.0,5079,82.3,598,9.7
4,1600000US0800925,"Akron town, Colorado",763,345,45.2,114,14.9,45,5.9,27,3.5,135,17.7,15,2.0,96,12.6,238,31.2,37,4.8,165,21.6,1.99,2.50,...,48.6,169,9.4,247,13.8,274,15.3,160,8.9,72,4.0,113,6.3,1412,78.8,1482,1482,761,51.3,721,48.7,1603,89.4,281,15.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,1600000US0886117,"Woodmoor CDP, Colorado",3224,2633,81.7,902,28.0,142,4.4,0,0.0,308,9.6,0,0.0,211,6.5,141,4.4,0,0.0,100,3.1,2.74,2.98,...,45.8,223,2.5,765,8.7,944,10.7,1232,13.9,1065,12.0,1123,12.7,6852,77.5,7069,7069,3811,53.9,3258,46.1,7970,90.2,236,2.7
478,1600000US0886200,"Woody Creek CDP, Colorado",69,42,60.9,0,0.0,0,0.0,0,0.0,27,39.1,0,0.0,27,39.1,0,0.0,0,0.0,0,0.0,1.64,2.05,...,38.9,0,0.0,0,0.0,50,44.2,0,0.0,0,0.0,11,9.7,113,100.0,113,113,69,61.1,44,38.9,89,78.8,24,21.2
479,1600000US0886310,"Wray city, Colorado",934,437,46.8,215,23.0,91,9.7,37,4.0,176,18.8,18,1.9,141,15.1,230,24.6,40,4.3,176,18.8,2.43,3.20,...,49.2,70,3.0,211,9.0,293,12.5,338,14.5,155,6.6,160,6.8,1605,68.6,1665,1665,857,51.5,808,48.5,2095,89.6,350,15.0
480,1600000US0886475,"Yampa town, Colorado",187,103,55.1,49,26.2,0,0.0,0,0.0,68,36.4,0,0.0,64,34.2,16,8.6,0,0.0,8,4.3,2.39,3.26,...,39.8,6,1.3,43,9.6,47,10.5,36,8.1,5,1.1,55,12.3,314,70.2,336,336,194,57.7,142,42.3,385,86.1,23,5.1


### Cleanup df_demographics
* several columns have no valid data unfortunately and can be dropped
* we will need to ensure each column has the correct associated data type for subsequent Folium/GeoJSON usage

In [15]:
def find_column_missing_all_values(row):
    """Helper function.  Effecient implementation for finding entire columns w/ missing values."""
    return row.values.all() == '-888888888'

def cast_type_columns(df):
    """Helper function.  Mapped columns follow a specific naming convention:
    ..._med for median, ..._mean for mean, ..._% for percent with predictable value types (int, float)
    params: df{Pandas DataFrame} - 
    returns: none - df is modifed in place
    """
    for col in df.columns[2:]:
        if any(suffix in col for suffix in ('_%', '_mean', '_avg', 'rate')):
            df[col] = df[col].astype('float')
        elif '_med' in col:
            df[col] = df[col].astype('int')
        else:
            df[col] = df[col].astype('int')
               
def cleanup_demographics_df(demographics_df):
    """Function to:
    1) drop blank columns (arrived blank from Census Bureau, unfortunately)
    2) cast type integers
    3) cast type floats
    params: demographics_df{Pandas DF} - dataframe to cleanup
    returns: demographics_df{Pandas DF} - cleaned dataframe
    """
    # cast string columns from Pandas object type - necessary for Folium/GeoJSON mapping later
    demographics_df.iloc[:, :2] = demographics_df.iloc[:, :2].astype(str)
    
    # the following columns are completely blank & can be dropped as cols convey no meaning
    col_missing_vals = df_demographics.columns[df_demographics.apply(find_column_missing_all_values)].to_numpy()
    demographics_df.drop(col_missing_vals, axis=1, inplace=True)
    
    # cast type int, float and all other columns
    cast_type_columns(demographics_df)
    
    return demographics_df


df_demographics = cleanup_demographics_df(df_demographics)
df_demographics

/tmp/ipykernel_400859/2061421237.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographics_df.drop(col_missing_vals, axis=1, inplace=True)
/tmp/ipykernel_400859/2061421237.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('int')
/tmp/ipykernel_400859/2061421237.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,GEO_ID,NAME,tot_hh,mar_hh,mar_hh_%,mar_child,mar_child_%,cohabit_hh,cohabit_hh_%,cohabit_child,cohabit_child_%,hh_male,hh_male_%,hh_male_child,hh_male_child_%,hh_male_solo,hh_male_solo_%,hh_female,hh_female_%,hh_female_child,hh_female_child_%,hh_female_solo,hh_female_solo_%,tot_hh_avg_hh_size,tot_hh_avg_fam_size,...,tot_pop_female_%,tot_pop_20-24,tot_pop_20-24_%,tot_pop_25-34,tot_pop_25-34_%,tot_pop_35-44,tot_pop_35-44_%,tot_pop_45-54,tot_pop_45-54_%,tot_pop_55-59,tot_pop_55-59_%,tot_pop_60-64,tot_pop_60-64_%,tot_pop_>=21,tot_pop_>=21_%,tot_pop_>=18,tot_pop_>=18_%,tot_pop_>=18_male,tot_pop_>=18_male_%,tot_pop_>=18_female,tot_pop_>=18_female_%,tot_pop_white,tot_pop_white_%,tot_pop_hisp,tot_pop_hisp_%
0,1600000US0800320,"Acres Green CDP, Colorado",1040,717,68.9,353,33.9,14,1.3,0,0.0,198,19.0,8,0.8,101,9.7,111,10.7,9,0.9,66,6.3,2.63,3.01,...,47.0,157,5.7,297,10.9,462,16.9,373,13.7,172,6.3,190,7.0,2067,75.7,2152,2152.0,1216,56.5,936,43.5,2221,81.3,63,2.3
1,1600000US0800620,"Aetna Estates CDP, Colorado",395,202,51.1,38,9.6,0,0.0,0,0.0,77,19.5,59,14.9,0,0.0,116,29.4,47,11.9,25,6.3,3.18,3.16,...,57.9,129,10.3,312,24.9,121,9.6,146,11.6,59,4.7,46,3.7,878,70.0,904,904.0,363,40.2,541,59.8,618,49.2,635,50.6
2,1600000US0800760,"Aguilar town, Colorado",271,56,20.7,0,0.0,32,11.8,25,9.2,37,13.7,0,0.0,26,9.6,146,53.9,0,0.0,129,47.6,1.76,2.71,...,63.1,51,10.7,8,1.7,35,7.3,70,14.7,40,8.4,53,11.1,415,87.0,451,451.0,164,36.4,287,63.6,322,67.5,323,67.7
3,1600000US0800870,"Air Force Academy CDP, Colorado",643,450,70.0,317,49.3,7,1.1,7,1.1,99,15.4,35,5.4,64,10.0,87,13.5,76,11.8,11,1.7,3.36,3.64,...,34.3,2727,44.2,429,7.0,375,6.1,178,2.9,74,1.2,6,0.1,2996,48.6,5174,5174.0,3516,68.0,1658,32.0,5079,82.3,598,9.7
4,1600000US0800925,"Akron town, Colorado",763,345,45.2,114,14.9,45,5.9,27,3.5,135,17.7,15,2.0,96,12.6,238,31.2,37,4.8,165,21.6,1.99,2.50,...,48.6,169,9.4,247,13.8,274,15.3,160,8.9,72,4.0,113,6.3,1412,78.8,1482,1482.0,761,51.3,721,48.7,1603,89.4,281,15.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,1600000US0886117,"Woodmoor CDP, Colorado",3224,2633,81.7,902,28.0,142,4.4,0,0.0,308,9.6,0,0.0,211,6.5,141,4.4,0,0.0,100,3.1,2.74,2.98,...,45.8,223,2.5,765,8.7,944,10.7,1232,13.9,1065,12.0,1123,12.7,6852,77.5,7069,7069.0,3811,53.9,3258,46.1,7970,90.2,236,2.7
478,1600000US0886200,"Woody Creek CDP, Colorado",69,42,60.9,0,0.0,0,0.0,0,0.0,27,39.1,0,0.0,27,39.1,0,0.0,0,0.0,0,0.0,1.64,2.05,...,38.9,0,0.0,0,0.0,50,44.2,0,0.0,0,0.0,11,9.7,113,100.0,113,113.0,69,61.1,44,38.9,89,78.8,24,21.2
479,1600000US0886310,"Wray city, Colorado",934,437,46.8,215,23.0,91,9.7,37,4.0,176,18.8,18,1.9,141,15.1,230,24.6,40,4.3,176,18.8,2.43,3.20,...,49.2,70,3.0,211,9.0,293,12.5,338,14.5,155,6.6,160,6.8,1605,68.6,1665,1665.0,857,51.5,808,48.5,2095,89.6,350,15.0
480,1600000US0886475,"Yampa town, Colorado",187,103,55.1,49,26.2,0,0.0,0,0.0,68,36.4,0,0.0,64,34.2,16,8.6,0,0.0,8,4.3,2.39,3.26,...,39.8,6,1.3,43,9.6,47,10.5,36,8.1,5,1.1,55,12.3,314,70.2,336,336.0,194,57.7,142,42.3,385,86.1,23,5.1


### Check column data types w/in dataframe

In [16]:
print(df_demographics.dtypes.value_counts(), '\n')
df_demographics.info()

float64    72
int64      70
object      2
Name: count, dtype: int64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Columns: 144 entries, GEO_ID to tot_pop_hisp_%
dtypes: float64(72), int64(70), object(2)
memory usage: 542.4+ KB


### Pickle demographics dataframe for later use

In [17]:
df_demographics.to_pickle('colorado_place_demographics.pkl')

## Occupations Associated with higher Alcohol Consumption
* This file serves to analyze occupations associated with higher than average alcohol consumption.
* We are able to search for these occupations given our Comparative Profile
* https://alcohol.org/professions/
* IMPORTANT NOTE: addiction is a serious issue and should be given proper consideration and treated accordingly.  This author assumes, the general American societal attitude, that every adult is free to make their own choices.

In [18]:
occ_dict = {
    'occupation': [
        'lawyers', 
        'nurses_and_healthcare_prof', 
        'mining', 
        'construction', 
        'hospitality_and_food_services', 
        'arts_and_entertainment', 
        'management', 
        'real_estate', 
        'finance_and_insurance', 
        'educational_services'
    ], 
    'alcohol_consumption_rate': [20, 15.3, 17.5, 16.5, 11.8, 11.5, 9.9, 5, 7.4, 4.7]
}

df_occ = pd.DataFrame(occ_dict)
df_occ.sort_values(by='alcohol_consumption_rate', ascending=False, inplace=True)

df_occ

,occupation,alcohol_consumption_rate
0,lawyers,20.0
2,mining,17.5
3,construction,16.5
1,nurses_and_healthcare_prof,15.3
4,hospitality_and_food_services,11.8
5,arts_and_entertainment,11.5
6,management,9.9
8,finance_and_insurance,7.4
7,real_estate,5.0
9,educational_services,4.7


## Related Occupations Found Within Census Bureau Demographics Data
* Census Bureau American Community Survey 2021 data; target occupations
    - Agg/foresty/fishing/hunting/mining
    - Construction
    - Prof., Scientific, mgm’t., admin, waste mgm’t.
    - Healthcare, education, social asst.
    - Arts, Entertainment, recreation, food services
* Since these occupations have been found to have higher rates of alcohol consumption we are interested in knowing if they occur anywhere within the approximate Denver-Lakewood-Aurora MSA